In [88]:
import pandas as pd
import csv

import os, os.path
import sys

# All chosen fields for survey 1
A grass hay field, "106433_WSDA_SF_2017", was missed from meeting PDF! how the hell did this happen?
- I had dropped grass hay from disagreement table! 

In [89]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choices_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

chosen_fields_set1 = pd.DataFrame()
for a_choice_sheet in choices_set_1_sheet_names:    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)
    chosen_fields_set1 = pd.concat([chosen_fields_set1, a_choice_sheet])

In [90]:
choices_set_2_xl = pd.ExcelFile(perry_dir + "set_2_handPicked.xlsx")
choices_set_2_sheet_names = choices_set_2_xl.sheet_names  # see all sheet names

chosen_fields_set2 = pd.DataFrame()
for a_choice_sheet in choices_set_2_sheet_names:    
    # read a damn sheet
    a_choice_sheet = choices_set_2_xl.parse(a_choice_sheet)
    chosen_fields_set2 = pd.concat([chosen_fields_set2, a_choice_sheet])

In [91]:
print (len(chosen_fields_set2.ID))
print (len(chosen_fields_set2.ID.unique()))

322
322


In [92]:
# # check if there are overlap between chosen fields in set 1 and set 2
# set1_uniqueIDs = list(chosen_fields_set1.ID.unique())
# set2_uniqueIDs = list(chosen_fields_set2.ID.unique())

# A = chosen_fields_set2[chosen_fields_set2.ID.isin(set1_uniqueIDs)]
# B = chosen_fields_set1[chosen_fields_set1.ID.isin(set2_uniqueIDs)]
# A.shape

# Expert Set 1 - post meeting

In [93]:
import pandas as pd

ML_data_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
expert_set_1_meeting_Consensus = pd.read_csv(ML_data_dir + "expert_set1_postmeeting_consensus.csv")
expert_set_1_meeting_Consensus.head(2)

,ID,Vote
0,99837_WSDA_SF_2017,2
1,114615_WSDA_SF_2017,1


In [94]:
print (expert_set_1_meeting_Consensus.shape)
print (expert_set_1_meeting_Consensus.Vote.unique())

(111, 2)
[2 1]


# Expert Set 1 - pre-meeting

In [95]:
expert_set1_premeeting_consensus = pd.read_csv(ML_data_dir + "expert_set1_premeeting_consensus.csv")
expert_set1_premeeting_consensus.head(2)

,ID,PerrysVote,AndrewsVote,TimsVote,PerryVCorrected,AndrewVCorrected,TimsVCorrected,Vote
0,157415_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,1
1,155601_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,1


In [96]:
drop_cols = ['PerrysVote', 'AndrewsVote', 'TimsVote',
             "PerryVCorrected", "AndrewVCorrected", "TimsVCorrected"]
expert_set1_premeeting_consensus.drop(labels=drop_cols, axis=1, inplace=True)

# Concatenate set 1 pre and post meeting labels

In [97]:
expert_set_1_labels = pd.concat([expert_set_1_meeting_Consensus, 
                                 expert_set1_premeeting_consensus[['ID', 'Vote']]])

# we need this shit, cuz some fields were requested to be discussed or had comment
expert_set_1_labels.drop_duplicates(inplace=True) 
expert_set_1_labels.shape

(285, 2)

# Read Experts' Set 2 pre-meeting consensus

In [98]:
expert_set2_premeeting_consensus = pd.read_csv(ML_data_dir + "expert_set2_premeeting_consensus.csv")
expert_set2_premeeting_consensus.head(2)

,ID,PerryV,AndrewV,PerryVCorrected,AndrewVCorrected,Vote
0,160820_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,1
1,161383_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,1


In [99]:
set1_IDs = list(expert_set_1_labels.ID.unique())

#
#    Get rid of repeated fields
#
expert_set2_premeeting_consensus=expert_set2_premeeting_consensus[~expert_set2_premeeting_consensus\
                                                                  .ID.isin(set1_IDs)]

In [100]:
expert_set2_premeeting_consensus.head(2)

,ID,PerryV,AndrewV,PerryVCorrected,AndrewVCorrected,Vote
0,160820_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,1
1,161383_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,1


# Non-Expert Labels Set-2

### Pre-Meeting Consensus

In [101]:
NE_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/nonExpert_set2_fields/"

NE_S2_PreMeeting=pd.read_csv(NE_dir+"NE_set2_premeeting_Consensus_IDs_Votes_IncludesUnsure.csv")
                                    "NE_set2_post_expert_meeting_IDs_Votes_IncludesNA_fromOneDrive_May23.csv"


# Drop the Unsure ones. We will read it in the post meeting part.
NE_S2_PreMeeting=NE_S2_PreMeeting[NE_S2_PreMeeting.HosseinV!="Unsure"]

# NE_S2_PreMeeting.rename(columns = {'HosseinV':'Vote'}, inplace = True)

NE_S2_PreMeeting["Vote"]=0
NE_S2_PreMeeting.loc[NE_S2_PreMeeting.HosseinV=="Single", "Vote"]=1
NE_S2_PreMeeting.loc[NE_S2_PreMeeting.HosseinV=="Double", "Vote"]=2

NE_S2_PreMeeting=NE_S2_PreMeeting[["ID", "Vote"]]

In [102]:
# Post-Meeting Consensus

(1753, 2)

# Concatenate DataFrames

In [17]:
train_labels = pd.concat([expert_set_1_labels, 
                          expert_set2_premeeting_consensus[['ID', 'Vote']],
                          NE_S2_PreMeeting])
train_labels.reset_index(drop=True, inplace=True)
# train_labels.drop_duplicates(inplace=True)
train_labels.shape

(1753, 2)

# Sep 8 2022

The train label we get from this notebook has 1753 fields in it. However, the one on the disk
has 1849. Probably somewhere else I have read the old train_labels.csv and added the new fields from somewhere to it.
But cannot recall now. Need to find it.

In [26]:
out_folder = "/Users/hn/Documents/01_research_data/NASA/ML_data/"

out_name = out_folder + "train_labels_2.csv"
train_labels.to_csv(out_name, index = False)

In [18]:
A = pd.read_csv("/Users/hn/Documents/01_research_data/NASA/ML_data/train_labels.csv")

In [30]:
len(A.ID.unique())

1849

In [31]:
evaluation_set = pd.read_csv("/Users/hn/Documents/01_research_data/NASA/parameters/evaluation_set.csv")

In [33]:
evaluation_set.head(2)

,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


In [35]:
A = pd.merge(A, evaluation_set, on=['ID'], how='left')

In [37]:
train_labels = pd.merge(train_labels, evaluation_set, on=['ID'], how='left')

In [41]:
A[A.ExctAcr>10].shape

(1342, 9)

In [42]:
train_labels[train_labels.ExctAcr>10].shape

(1267, 9)

In [46]:
expert_set_1_labels=pd.merge(expert_set_1_labels, evaluation_set, on=['ID'], how='left')
expert_set2_premeeting_consensus=pd.merge(expert_set2_premeeting_consensus, evaluation_set, on=['ID'], how='left')
NE_S2_PreMeeting=pd.merge(NE_S2_PreMeeting, evaluation_set, on=['ID'], how='left')

In [49]:
NE_S2_PreMeeting.shape

(1319, 9)

In [50]:
expert_set2_premeeting_consensus.shape

(149, 13)

In [51]:
expert_set_1_labels.shape

(285, 9)

In [53]:
A.shape

(1849, 9)

In [56]:
train_labels.head(2)

,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,99837_WSDA_SF_2017,2,barley,center pivot,wsda,73,73.250981,2017/10/10,Grant
1,114615_WSDA_SF_2017,1,barley,center pivot,wsda,31,30.993504,2017/10/10,Grant


In [57]:
A_inside_train_labels = A[A.ID.isin(list(train_labels.ID))]

In [59]:
A_inside_train_labels.shape

(1753, 9)

In [60]:
A.shape

(1849, 9)

In [61]:
train_labels_in_A = train_labels[train_labels.ID.isin(A.ID)]

In [62]:
train_labels_in_A.shape

(1753, 9)

In [63]:
train_labels.shape

(1753, 9)

In [64]:
A_out_of_train_labels=A[~A.ID.isin(list(train_labels.ID))]

In [65]:
A_out_of_train_labels.shape

(96, 9)

In [66]:
A_out_of_train_labels

,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
1753,107807_WSDA_SF_2018,2,barley hay,center pivot,wsda,57,56.853141,2018/06/05 00:00:00,Franklin
1754,107169_WSDA_SF_2017,2,"bean, dry",center pivot,wsda,17,17.149065,2017/10/19,Grant
1755,109777_WSDA_SF_2017,2,"bean, dry",center pivot,wsda,140,140.014732,2017/07/18,Grant
1756,103840_WSDA_SF_2018,2,"bean, green",center pivot,wsda,11,11.168004,2018/09/12 00:00:00,Franklin
1757,103204_WSDA_SF_2018,1,bluegrass seed,center pivot,wsda,5,5.351631,2018/09/11 00:00:00,Franklin
...,...,...,...,...,...,...,...,...,...
1844,57406_WSDA_SF_2016,1,wheat,center pivot,wsda,88,87.734070,2016/09/14 00:00:00,Adams
1845,8195_WSDA_SF_2016,1,wheat,center pivot,wsda,52,52.328133,2016/08/31 00:00:00,Benton
1846,98334_WSDA_SF_2017,2,wheat,center pivot,wsda,151,151.076270,2017/06/27,Grant
1847,99141_WSDA_SF_2017,2,wheat,center pivot,wsda,116,116.234894,2017/10/17,Grant


In [79]:
NE_S2_PreMeeting_in_train_labels=NE_S2_PreMeeting[NE_S2_PreMeeting.ID.isin(
                                  list(train_labels.ID))]
print (NE_S2_PreMeeting_in_train_labels.shape)
print (NE_S2_PreMeeting.shape)

(1319, 9)
(1319, 9)


In [81]:
expert_set2_premeeting_consensus_labels_in_train=expert_set2_premeeting_consensus[
     expert_set2_premeeting_consensus.ID.isin(list(train_labels.ID))]
print (expert_set2_premeeting_consensus_labels_in_train.shape)
print (expert_set2_premeeting_consensus.shape)

(149, 13)
(149, 13)


In [82]:
train_labels.shape

(1753, 9)

In [85]:
len(A.ID.unique())
len(A.ID)

1849

In [86]:
len(train_labels.ID.unique())
len(train_labels.ID)

1753